# Decorators

## What is a decorator
A decorator is a way to wrap a function to add functionality before and after a metod is called or to augment a type.

A simple wrapper:

In [20]:
def very_simple_decorator(f):
    def simple_wrapper():
        print("Entering function.")
        result = f()
        print ("Leaving function.")
        return result
    return simple_wrapper

@very_simple_decorator
def say_hi():
    print("Well, Hello there!")

@very_simple_decorator
def say_goodbye():
    print("toodles!")

say_hi()
say_goodbye()


Entering function.
Well, Hello there!
Leaving function.
Entering function.
toodles!
Leaving function.


This is a more complex wrapper that passes arguments along:

In [32]:
import functools

def better_decorator(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        print("Entering function.")
        result = f(*args,**kwargs)
        
        print (f"Result is {result}.")
        return result
    return wrapper

@better_decorator
def square(x):
    return x**2



@better_decorator
def add_two(x):
    return x+2

answer = add_two(3)
answer = square(5)
print(answer)

Entering function.
Result is 5.
Entering function.
Result is 25.
199


This works really well to time your functions, or for debugging..

In [27]:
import functools
import time
debug_on = True

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        if debug_on == True:
            args_repr = [repr(a) for a in args]                      # 1
            kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
            signature = ", ".join(args_repr + kwargs_repr)           # 3
            print(f"Calling {func.__name__}({signature})")
            tic = time.perf_counter()
        value = func(*args, **kwargs)
        if debug_on == True:
            toc = time.perf_counter()
            print(f"{func.__name__!r} returned {value!r}")           # 4
            print(f"The function took {toc-tic:0.8f} seconds to run.")
        return value
    return wrapper_debug

@debug
def squared(x:int):
    return x**2

squared(4)

debug_on = False
squared(4)

Calling squared(4)
'squared' returned 16
The function took 0.00000188 seconds to run.


16

## Decorators with parameters
Sometimes it's helpful to create decorators with parameters.  

In [29]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")

greet("bob")

Hello bob
Hello bob
Hello bob


## Class decorator

Sometimes it's helpful to decorate a class:


In [33]:
from dataclasses import dataclass
from math import sqrt

@dataclass
class Coordiates:
    x:int
    y:int

c = Coordiates(1,2)

print (sqrt(c.x**2+c.y**2))



2.23606797749979


## Helpful built-in decorators

- @abstract -- marks a class or its methods abstract.
- @staticmethod -- makes a method static in a class
- @property -- creates a property from a variable.
- @classmethod -- makes a method a class method.
- @lru_cache -- caches the results of a function. Use those results if the same parameters are passed later.
- @contextmanager -- used for defining classes that manage context. (future lesson)
- @cached_property


In [34]:
from functools import cached_property


class Circle:
    def __init__(self, radius):
        self.radius = radius

    @cached_property
    def area(self):
        return 3.14 * self.radius ** 2


circle = Circle(10)
print(circle.area)
# prints 314.0
print(circle.area)
# returns the cached result (314.0) directly

314.0
314.0
